In [1]:
from datasets import load_dataset
#from googletrans import Translator
from tqdm import tqdm
from llm.GepetoBase import ChatGPT
import pandas as pd
import numpy as np

In [2]:
# Função para traduzir o texto de inglês para português
gpt = ChatGPT('../key.txt') 

def traduzir_texto(chat_gpt, prompt):
    
    chat_gpt.add_message(role="user", prompt=prompt)

    # fazendo inferencia
    resposta, _ = chat_gpt.get_completion()

    chat_gpt.reset_messages(maintain_context=False)
    
    return resposta

In [4]:
# estrutura para armazenar os dados processados. Cada elemento da lista é uma lista com 8 elementos: dataset do benchmark, pergunta, input, resposta, letra A, letra B, letra C e letra D.
dados = []

# Pré-processando o dataset MedMCQA

In [5]:
medmcqa = load_dataset('medmcqa')['validation']

In [6]:
for index in tqdm(range(len(medmcqa))):
    pergunta = medmcqa[index]['question']+"\n"
    pergunta+=f"A){medmcqa[index]['opa']}\n"
    pergunta+=f"B){medmcqa[index]['opb']}\n"
    pergunta+=f"C){medmcqa[index]['opc']}\n"
    pergunta+=f"D){medmcqa[index]['opd']}\n"
    
    correta = ['A','B', 'C', 'D']
    
    
    prompt = f"Você irá receber uma questão de múltipla escolha, em inglês, de modo que a você será passado o enunciado, junto com as alternativas, demarcadas por 'A)', 'B)', 'C)' e 'D)'. Você deve traduzir a pergunta e as suas alternativas para o português. Ao traduzir a pergunta, mantenha as alternativas 'A)', 'B)', 'C)' e 'D)', com suas respectivas traduções, não fuja desse formato. Também, fique atento a possíveis abreviações, expressões populares e polissemias, e traduza da melhor forma possível para o português, removendo possíveis ruídos do texto também. A pergunta/questão está marcada com a tag <pergunta>.\n Retorne a tradução da pergunta e de suas alternativas em um json, de modo que a pergunta e as alternativas traduzidas estejam em chaves 'pergunta' e 'A', 'B', 'C', 'D' para as alternativas. Segue um exemplo, a ser seguido por você, de retorno: '{{'pergunta':'enunciado da questao', 'A': 'letra A', 'B': 'letra B', 'C': 'letra C', 'D': 'letra D'}}'.\n\n <pergunta> {pergunta} <pergunta> \n."
    
    
    try:
        
        resposta = traduzir_texto(gpt, prompt).replace('\n', ' ')
    
        if resposta[-1] != "}":
            resposta = resposta[:-1]
            
        resp = eval(resposta)
        
        dados.append(['MedMCQA', resp['pergunta'], np.nan, correta[medmcqa[index]['cop']], resp['A'], resp['B'], resp['C'], resp['D']])
        
    except SyntaxError:
        
        continue
    
    except:
        gpt.reset_messages(maintain_context=False)
        resposta = traduzir_texto(gpt, prompt[:int(len(prompt)*0.75)]).replace('\n', ' ')
    
        if resposta[-1] != "}":
            resposta = resposta[:-1]
        
        try:
            resp = eval(resposta)
            
            dados.append(['MedMCQA', resp['pergunta'], np.nan, correta[medmcqa[index]['cop']], resp['A'], resp['B'], resp['C'], resp['D']])
            
        except:
            
            continue


100%|██████████| 4183/4183 [4:01:04<00:00,  3.46s/it]     


# Pré-processando o dataset MedQA(USMLE)

In [7]:
medqa = load_dataset('GBaker/MedQA-USMLE-4-options')['test']

In [8]:
for index in tqdm(range(len(medqa))):
    
    pergunta = medqa[index]['question']+"\n"
    pergunta+=f"A){medqa[index]['options']['A']}\n"
    pergunta+=f"B){medqa[index]['options']['B']}\n"
    pergunta+=f"C){medqa[index]['options']['C']}\n"
    pergunta+=f"D){medqa[index]['options']['D']}\n"
    
    prompt = f"Você irá receber uma questão de múltipla escolha, em inglês, de modo que a você será passado o enunciado, junto com as alternativas, demarcadas por 'A)', 'B)', 'C)' e 'D)'. Você deve traduzir a pergunta e as suas alternativas para o português. Também, fique atento a possíveis abreviações, expressões populares e polissemias, e traduza da melhor forma possível para o português, removendo possíveis ruídos do texto também. A pergunta/questão está marcada com a tag <pergunta>.\n Retorne a tradução da pergunta e de suas alternativas em um json, de modo que a pergunta e as alternativas traduzidas estejam em chaves 'pergunta' e 'A', 'B', 'C', 'D' para as alternativas. Segue um exemplo, a ser seguido por você, de retorno: '{{'pergunta':'enunciado da questao', 'A': 'letra A', 'B': 'letra B', 'C': 'letra C', 'D': 'letra D'}}'. Não fuja desse formato, sempre crie uma chave para cada alternativa no dicionário, só retorne a resposta no formato do exemplo.\n\n <pergunta> {pergunta} <pergunta> \n."
    
    try:
        
        resposta = traduzir_texto(gpt, prompt).replace('\n', ' ')
    
        if resposta[-1] != "}":
            resposta = resposta[:-1]
            
        resp = eval(resposta)
        
        dados.append(['MedQA (USMLE)', resp['pergunta'], np.nan, medqa[index]['answer_idx'], resp['A'], resp['B'], resp['C'], resp['D']])
        
    except SyntaxError:
        
        continue
    
    except:
        gpt.reset_messages(maintain_context=False)
        resposta = traduzir_texto(gpt, prompt[:int(len(prompt)*0.75)]).replace('\n', ' ')
    
        if resposta[-1] != "}":
            resposta = resposta[:-1]
        
        try:
            resp = eval(resposta)
            
            dados.append(['MedQA (USMLE)', resp['pergunta'], np.nan, medqa[index]['answer_idx'], resp['A'], resp['B'], resp['C'], resp['D']])
            
        except:
            
            continue    

100%|██████████| 1273/1273 [3:44:12<00:00, 10.57s/it]   


# Pré-processando o dataset PubMedQA

In [9]:
pubmedqa = load_dataset('hippocrates/pubmedqa_test')['test']

In [10]:
for index in tqdm(range(len(pubmedqa))):
    
    instruction = pubmedqa[index]['query'].split('INPUT')[0] + pubmedqa[index]['query'].split('INPUT')[1][2:].split(']')[1].split('OUTPUT')[0].strip()
    
    input = pubmedqa[index]['query'].split('INPUT')[1][2:].split(']')[0][1:]
    
    output = pubmedqa[index]['answer']
    
    prompt = f"Você irá receber uma pergunta, em inglês, relacionada ao tema da saúde. Além disso, você irá receber dados adicionais que servem para auxiliar na pergunta e também irá receber a resposta para a questão. Você deve traduzir a pergunta, os dados adicionais e a resposta para o português. Também, fique atento a possíveis abreviações, expressões populares e polissemias, e traduza da melhor forma possível para o português, removendo possíveis ruídos do texto também. A pergunta está marcada com a tag <pergunta>, os dados adicionais estão marcados com <info> e a resposta está marcada com <resposta>.\n Retorne a tradução da pergunta, dados adicionais e resposta em um json, de modo que a pergunta, dados adicionais e resposta estejam em chaves 'pergunta', 'info', 'resposta', respectivamente. Segue um exemplo, a ser seguido por você, de retorno: '{{'pergunta':'enunciado da pergunta', 'info': 'dados adicionais', 'resposta':'resposta traduzida'}}'.\n\n <pergunta> {instruction} <pergunta>\n\n <info> {input} <info>\n\n<resposta> {output} <resposta> \n."
    
    
    try:
        
        resposta = traduzir_texto(gpt, prompt).replace('\n', ' ')
    
        if resposta[-1] != "}":
            resposta = resposta[:-1]
            
        resp = eval(resposta)
        
        dados.append(['PubMedQA', resp['pergunta'], resp['info'], resp['resposta'], np.nan, np.nan, np.nan, np.nan])
        
    except SyntaxError:
        
        continue
    
    except:
        gpt.reset_messages(maintain_context=False)
        resposta = traduzir_texto(gpt, prompt[:int(len(prompt)*0.75)]).replace('\n', ' ')
    
        if resposta[-1] != "}":
            resposta = resposta[:-1]
        
        try:
            resp = eval(resposta)
            
            dados.append(['PubMedQA', resp['pergunta'], resp['info'], resp['resposta'], np.nan, np.nan, np.nan, np.nan])
            
        except:
            
            continue

100%|██████████| 500/500 [1:03:52<00:00,  7.67s/it]


# Pré-processando o dataset MMLU

In [11]:
subjects = ['anatomy', 'clinical_knowledge', 'college_medicine', 'medical_genetics', 'professional_medicine', 'college_biology']

In [12]:
correta = ['A','B', 'C', 'D']
for subject in subjects:
    
    mmlu = load_dataset('cais/mmlu',subject)['test']

    for index in tqdm(range(len(mmlu))):
        
            
        pergunta = mmlu[index]['question']+"\n"
        pergunta+=f"[Alt.A]{mmlu[index]['choices'][0]}\n"
        pergunta+=f"[Alt.B]{mmlu[index]['choices'][1]}\n"
        pergunta+=f"[Alt.C]{mmlu[index]['choices'][2]}\n"
        pergunta+=f"[Alt.D]{mmlu[index]['choices'][3]}\n"
        
        prompt = f"Você irá receber uma questão de múltipla escolha, em inglês, de modo que a você será passado o enunciado, junto com as alternativas, demarcadas por 'A)', 'B)', 'C)' e 'D)'. Você deve traduzir a pergunta e as suas alternativas para o português. Ao traduzir a pergunta, mantenha as alternativas 'A)', 'B)', 'C)' e 'D)', com suas respectivas traduções, não fuja desse formato. Também, fique atento a possíveis abreviações, expressões populares e polissemias, e traduza da melhor forma possível para o português, removendo possíveis ruídos do texto também. A pergunta/questão está marcada com a tag <pergunta>.\n Retorne a tradução da pergunta e de suas alternativas em um json, de modo que a pergunta e as alternativas traduzidas estejam em chaves 'pergunta' e 'A', 'B', 'C', 'D' para as alternativas. Segue um exemplo, a ser seguido por você, de retorno: '{{'pergunta':'enunciado da questao', 'A': 'letra A', 'B': 'letra B', 'C': 'letra C', 'D': 'letra D'}}'.\n\n <pergunta> {pergunta} <pergunta> \n."
    
    
        try:
            
            resposta = traduzir_texto(gpt, prompt).replace('\n', ' ')
        
            if resposta[-1] != "}":
                resposta = resposta[:-1]
                
            resp = eval(resposta)
            
            dados.append([f'MMLU/{subject}', resp['pergunta'], np.nan, correta[mmlu[index]['answer']], resp['A'], resp['B'], resp['C'], resp['D']])
            
        except SyntaxError:
            
            continue
        
        except:
            gpt.reset_messages(maintain_context=False)
            resposta = traduzir_texto(gpt, prompt[:int(len(prompt)*0.75)]).replace('\n', ' ')
        
            if resposta[-1] != "}":
                resposta = resposta[:-1]
            
            try:
                resp = eval(resposta)
                
                dados.append([f'MMLU/{subject}', resp['pergunta'], np.nan, correta[mmlu[index]['answer']], resp['A'], resp['B'], resp['C'], resp['D']])
                
            except:
                
                continue
        
        

100%|██████████| 144/144 [09:08<00:00,  3.81s/it]


# Reunindo os dados captados em um DataFrame

In [13]:
import pandas as pd

In [118]:
df = pd.DataFrame(dados, columns=['dataset', 'instruction', 'input', 'output', 'alternative_a', 'alternative_b', 'alternative_c', 'alternative_d'])

In [123]:
df.shape

(7045, 8)

In [124]:
df

,dataset,instruction,input,output,alternative_a,alternative_b,alternative_c,alternative_d
0,MedMCQA,Qual das seguintes afirmações não é verdadeira...,NaN,A,O impulso através das fibras mielinizadas é ma...,Correntes de membrana são geradas nos nós de R...,A condução saltatória dos impulsos é observada,A anestesia local é eficaz apenas quando o ner...
1,MedMCQA,Qual das seguintes afirmações não é verdadeira...,NaN,A,A pressão oncótica do fluido que sai dos capil...,A concentração de glicose nos capilares é a me...,A constrição da arteríola aferente diminui o f...,O hematócrito do fluido que sai dos capilares ...
2,MedMCQA,Uma mulher de 29 anos com uma gravidez de 17 s...,NaN,C,"Não é necessário nenhum teste agora, pois sua ...",O ultrassom neste momento definitivamente dirá...,Amostras de líquido amniótico mais análise cro...,A triagem sanguínea neste momento esclarecerá ...
3,MedMCQA,O transporte axonal é:,NaN,C,Anterógrado,Retrogrado,Anterógrado e retrogrado,Nenhum
4,MedMCQA,A relação baixa de insulina para glucagon é vi...,NaN,A,Síntese de glicogênio,Quebra de glicogênio,Gluconeogênese,Cetogênese
...,...,...,...,...,...,...,...,...
7040,PubMedQA,Tarefa: Sua tarefa é responder a perguntas bio...,['Francófonos podem experimentar uma saúde mai...,talvez,NaN,NaN,NaN,NaN
7041,PubMedQA,Tarefa: Sua tarefa é responder a perguntas bio...,['Feedback dos resultados é o processo de apre...,talvez,NaN,NaN,NaN,NaN
7042,MMLU/clinical_knowledge,o que é 'coring'?\n,NaN,D,um sintoma de flebite.\n,titulação de uma droga com o efeito desejado.\n,a entrega da medicação diretamente na corrent...,quando os medicamentos são retirados de um fr...
7043,MMLU/college_medicine,para determinar a mudança de doppler na frequê...,NaN,D,eu apenas\n,i e iii\n,ii e iv\n,i e iv


In [121]:
df.dataset.value_counts()

dataset
MedMCQA                       4183
MedQA (USMLE)                 1273
PubMedQA                       500
MMLU/professional_medicine     272
MMLU/clinical_knowledge        265
MMLU/college_medicine          173
MMLU/college_biology           144
MMLU/anatomy                   135
MMLU/medical_genetics          100
Name: count, dtype: int64

In [125]:
df.to_csv('../dados/teste/drbyte_ptbr_teste.csv', index=False)